<img src="https://drive.google.com/uc?id=1Oj6yPUd6IG1yoNg1F-wGZlwVAJesgCi7" width="210" alt="" style="float:right;margin:15px;">

# Week 11 - Sensitivity Analysis & Design of Experiments
**This short notebook is a first try towards some more detailed course/reader material in the course [41365 Robust Design of products & mechanisms](https://kurser.dtu.dk/course/41635) at the [Technical University of Denmark](https://www.dtu.dk/). It can be distributed under the   terms of the [Creative Commons Attribution-ShareAlike licence](https://creativecommons.org/licenses/by-sa/4.0/).**

**Author:** [Tobias Eifler](https://orbit.dtu.dk/en/persons/tobias-eifler) (<tobeif@mek.dtu.dk>)  


This notebook contains some simple Python scripts using the libraries below. While not needed for the course at all, you are of course welcome to consult the corresponding documentation for the pretty standard packages numpy, scipy, pandas etc. (links to follow).


In [ ]:
# import of necessary modules (needs to be run before starting on the code cells below!)

import numpy as np
import scipy.stats
import pandas
import matplotlib.pyplot as plt
%matplotlib inline
import sympy

# Part 1 - _Sensitivity Analysis_
Sensitivity measures play an important role in model building and corroboration as well as parameter prioritization (for some more details see for example [Frey & Patil (2002)](https://doi.org/10.1111/0272-4332.00039) / for a comprehensive overview [Saltelli et al. (2008)](http://www.andreasaltelli.eu/file/repository/A_Saltelli_Marco_Ratto_Terry_Andres_Francesca_Campolongo_Jessica_Cariboni_Debora_Gatelli_Michaela_Saisana_Stefano_Tarantola_Global_Sensitivity_Analysis_The_Primer_Wiley_Interscience_2008_.pdf)). 

In the field of Robust Design, typical sensitivity measures can also be used as the simplest form of robustness metrics and are a well-established way to relate the change of an independent variable to the change of a dependent variable. In the context of a design task, this relates to predicting how functional performance (dependent/output variables) is affected by changes in the design paramters or potential noise factors (independent/input) variables. In the course, this has previously been related to the concept of the *transfer function* (see course material from [Week 2](https://learn.inside.dtu.dk/))

<img src="https://drive.google.com/uc?id=1fhS3r0f6BYl5sY2xHTKlq4S9-kc9BGY4" width=400px>

Usually, functional performance is affected by a number of design parameters. Therefore it can be important to determine which design parameter affects the functional performance the most. Sensitivity analysis consequently helps (1) to either identify relevant influences from a list of all factors affecting the functional performance, or (2) to compare different solutions in terms of their robustness towards specific influences. This can be used for decisive decision-making throughout product development, because it creates transparency with regards to which design parameters that have the greatest affect on the function and hereby the robustness.

**Definition:** 
Sensitivitiy analysis is the “study of how variation in the output of a model (numerical or otherwise) can be apportioned to different sources of uncertainty in the model input” 

**Objectives of a sensitivity analysis:**
- Assessing the importance and the relation between measurable variables in (complex) systems,
- Prioritization of influencing factors to deepen an analysis, for factor fixing in order to reduce model complexity,
- to compare different design solutions, to allow for informed decisions, etc.

In the simplest case, sensitivity is investigated based on an available data set or an available analytical function that creates the relation between input variables/parameters and the functional performance of a product. This case is assumed in the following, and besides some general infomration, some corresponding methods such as simple [visual sreening](#cont:screening), [nominal-range sensitivity](#cont:nominal), [derivatives](#cont:derivatives), and [regression analysis](#cont:regression) are presented.  

<a id='cont:screening'></a>
## An initial comparison - visual screening:
The usual starting point for an analysis is often the initial visual inspection of potential influences. An example is the simple inspection of available data sets in for example scatter plots, which is presented in the code below.

Give it a try and assess to which of the influences $x_1$ or $x_2$ the response value $y$ reacts more sensitive.

In [ ]:
# Two different datatesets

x1 = [3,2,2,3,1,1,2,3,4,1,4]
y1 = [109,102,102,98,85,95,96,98,123,94,102]

x2 = [5,7,8,7,2,10,2,9,4,11,12]
y2 = [91,86,87,88,89,87,92,87,92,95,90]

# Plotting the corresponding scatter(plots)

plt.rcParams['figure.figsize'] = [12, 7]

plt.subplot(1, 2, 1) # row 1, col 2 index 1
plt.scatter(x1, y1)
#plt.yticks(np.arange(85, 120, 5.0))
plt.xlabel("$x_1$")
plt.ylabel("$y$")

plt.subplot(1, 2, 2) # index 2
plt.scatter(x2, y2)
#plt.yticks(np.arange(85, 120, 5.0))
plt.xlabel("$x_2$")
plt.ylabel("$y$")
plt.show()


While the above exercise is straight forward, one important aspect of this initial comparison is of course choosing an equal scale for the axes in your plots. Therefore, please re-run the code after defining the same scale for the $y-$axis in the above example (uncomment the ```plt.yticks(np.arange(...)``` command by deleting the ```#``` symbol).

At the same time, this initial visual comparison of sensitivity can of course be based on scatterplots of available data sets in the same way as it can be based on a comparison of an available transfer function. You can try this out in the code below, which basically plots a function that you define. Initially the code includes:

$f_1(x_1,x_2)=x_1^2$

$f_2(x_1,x_2)=x_1^2+x_2$

Before running the code, make a reasonable guess which of the input factors the two cases are most sensitive to. 

In [ ]:
def fct1(x1, x2):
    return (x1**2)

def fct2(x1, x2):
    return (x1**2)+x2

from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = [7, 7]

N = 100

ax = plt.figure().add_subplot(projection="3d")
x = np.linspace(-2, 2, N)
y = np.linspace(0, 4, N)
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, fct1(X, Y), alpha=0.8)
ax.set_facecolor("white")
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")

ax = plt.figure().add_subplot(projection="3d")
x = np.linspace(-2, 2, N)
y = np.linspace(0, 4, N)
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, fct2(X, Y), alpha=0.8)
ax.set_facecolor("white")
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");


Is your prediction correct, or, more importantly, does your prediction hold true for all design points in the plot? What is for example the difference in sensitivity to the two input variables $x_1, x_2$ at point $(0, 0)$ vs. $(-1.5, 0)$? And lastly, do you see these difference also when repeating the simple exercise with self-defined functions ```def fct1(x1, x2)``` and ```def fct2(x1, x2)``` in the code above. 

**Short intermediate conclusions**: in essence, there should be two aspects to remember from the exercise
1. You can inspect sensitivity visually based on available data, or also just based on an available analytical description of the transfer function,
2. Sensitivity analyses are often conducted locally, i.e. depending on the design point. In other words, a given transfer function might not necessarily be more/less sensitivte over the full design range.


## Classification of sensitivity methods
for more detail on classifications, see for example [Frey & Patil (2002)](https://doi.org/10.1111/0272-4332.00039) or in the field of Robust Design also [Göhler & Eifler (2002)](https://doi.org/10.1115/1.4034112).

### Mathematical methods (local)
*Mathematical methods* assess sensitivity of a model output to a range of variation of the model input, in other words these methods typically rely on simply calculating the output for some few input values that represent the possible range of the input. The methods consequently do not address the actual variance of the input variables! Instead they assess the impact of an assumed range of variation in the input values on the output. Due to their simplicity, mathematical methods are mainly used for screening the most important inputs. In a Robust Design context, this includes a comparison of different input variables in terms of sensitivity, the identification of inputs that require further data acquisition or research, or also the initial comparison of different solutions when the main noise factors are know.

Mathematical methods are typically local sensitivity measures as they only address a small portion of the possible space of input values because interactions among inputs are difficult to capture!

### Statistical methods (local & global)
Statistical methods involve running simulations in which inputs are assigned probability distributions, and assessing the effect of variance in inputs on the output distribution. Depending on the method, one or more inputs are varied at a time so that interactions among multiple inputs can also be captured. The range and relative likelihood of inputs can be propagated using a variety of techniques such as Monte Carlo simulation, Latin hypercube sampling, and other methods.

While many statistical methods can also be considered as local $\it\color{red}{\text{(methods treated in the course are local sensitivity measures!)}}$, there are sophisticated statistical approaches such as variance decomposition techniques that allow for a global sensitivity analysis. In this a sensitivity analysis is considered global when all inputs are considered as simultaneously varying, and when the sensitivity is evaluated over the entire range of all inputs.


## Exemplary methods in the course

<a id='cont:nominal'></a>
### 1. Nominal Range Sensitivity
The nominal range sensitivity method, also known as threshold analysis, is usually used for screening purposes, that is to identify the most important inputs that propagate through a given model. A nominal range sensitivity analysis evaluates the effect of individually varying only one of the model inputs across a predefined set of values (usually a percentage value) on the model output. That means all other inputs are held at their nominal or base-case values (also reffered to as a *One-factor at a time approach*). The difference in the model output due to the change in the input variable is referred to as the sensitivity to that particular input variable. In most cases, the sensitivity is furthermore normalised with the nominal value, i.e. given as a percentage change compared to the nominal solution, in order to account for size dependency of tolerances for example. 

The (normalised) sensitivity coefficient can be calculated as:

$\large\phi = \frac{f(x_2)-f(x_1) / f(x_0)} {(x_2 - x_1) / x_0}$

where $x_0$ describes the nominal value, e.g. of a single design variable $DP_i$. Or also in more general terms:

$\large\phi = \frac{(f(x_0+\Delta)-f(x_0))/f(x_0)}{\Delta}$

where $\Delta$ can be a simple percentage value in positive or negative direction.


***Advantages:*** simple method that is easily applied and works well with linear models. Can be extended to a contribution analysis when there is good idea of plausible ranges that can be assigned to each selected input (i.e. considering the actual variation for individual inputs instead of a overall variation assumption as general percetage value for all inputs). 

***Disadvantages:*** potentially misleading results for nonlinear models with larger input variation, and in case of significant interactions between the input factors. For other than linear models, it is not given that nominal range sensitivity can provide a fully reliable rank ordering of key inputs.


<a id='cont:derivative'></a>
### 2. Differential sensitivity analysis
When generalising the above formulations, i.e. for the limit of the interval $\Delta \rightarrow 0$, the expression yields the formal definition of the derivative of a function $f$ towards a variable $x$:

$f'(x)=\lim_{x \to 0}\frac{f(x+\Delta)-f(x)}{\Delta}$

or in the case of multiple independent variables, the partial derivatives:

$\frac{\partial f(X)}{\partial x_i}=\lim_{x \to 0}\frac{f(x_1,...,x_i+\Delta,...,x_n)-f(x)}{\Delta}$

Derivatives are correspondingly one of the most used, and often easiest to apply sensitivity indices, if the function is not excessivly difficult to differentiate at least. However, please be aware that a simple sensitivity analysis based on derivatives basically implies the same advantages and disadvantages as the nominal range sensitivity above, particularly the limitations to linear functions and small variation windows. For a choice between both methods, it is therefore mostly about the necessary assumptions regarding pre-knowledge of transfer function, design-point and/or tolerance range.

As an example for a differential sensitivity analysis, we go back to our function $f_2(x_1,x_2)=x_1^2+x_2$ from above. After having plotted the function to get a feel for its shape, we can  undertake a local sensitivity analysis by calculating the local derivatives with respect to the two input parameters. In this case it is of course easy to differentiate by hand, but we use the example to demonstrate the use of the SymPy library to do symbolic differentiation with the computer.

In [ ]:
from sympy.interactive import printing
printing.init_printing(use_latex="mathjax")

#Calculating and printing the partial derivatives

x1 = sympy.Symbol("x1")
x2 = sympy.Symbol("x2")
fct2=x1**2+x2
d1 = sympy.diff(fct2, x1)
print('Partial derivative to x_1:')
d1  

In [ ]:
d2 = sympy.diff(fct2, x2)
print('Partial derivative to x_2:')
d2 

And after the initial visual inspection of the function above, the derivatives of course make sense. While the sensitivity changes across the input range for $x_1$, it stays constant for $x_2$. As discussed, a local sensitivity measure only provides a completely reliable result for linear functions, while non-linearity requires us to actually consider the design point and the corresponding error when the range of variation increases. 

For the given function, it seems that it has flat area around $(0, 0)$ with an increasing sensitivity towards the sides, e.g. around $(−2,−2)$ where the function looks much steeper. In other word, the function will have a higher local sensitivity, which is checked below by calculating $\frac{∂f}{∂x_1}(0, 0)$, $\frac{∂f}{∂x_2}(0, 0)$, $\frac{∂f}{∂x_1}(-2, -2)$, and $\frac{∂f}{∂x_2}(-2, -2)$.

In [ ]:
# Subs(tituting) the actual values in the above derivatives and printing of results

print('Partial derivative to x1 around design point (0,0):', d1.subs({x1: 0, x2: 0}))
print('Partial derivative to x2 around design point (0,0):', d2.subs({x1: 0, x2: 0}))
print('Partial derivative to x1 around design point (-2,-2):', d1.subs({x1: -2, x2: -2}))
print('Partial derivative to x2 around design point (-2,-2):', d2.subs({x1: -2, x2: -2}))

**Exercise:** You are given a beam structure that you should investigate based on the above sensitivity measures. 

<img src="https://drive.google.com/uc?id=1-IniZfZZJK3brhxP-i1yVjOY5I5rGMQw" width=500px>

First of all, please make an initial screening based on plotting the given transfer function (is the provided example a suitable one? If not, please explain what you'd do differently?). Then, please calculate the nominal range sensitivity index for all variables (with an increment of $\Delta=+1\%$) and compare it to the corresponding derivatives around the nominal design point. Is there a difference, and would this difference change for other regions of the design space?

While intendend as small programming exercise (**you have all necessary commands given in the content of this notebook**), you are of course also free to calculate the exercise by hand.

In [ ]:
#Define the Input nominal size and variation (sigma value) 
F  = 200
h  = 15
b  = 10 
L  = 1000

# Define fuction for calculating the resulting stress
def BeamStress(F,L,b,h):
    return 6*F*L/(b*h**2)

#calculate stress at the given nominal values 
print('Stress at the intended Design Point:',BeamStress(F,L,b,h))

# First inital screening based on plotting a small number of dimensions

from mpl_toolkits.mplot3d import Axes3D

N = 10
fig = plt.figure()
ax = plt.subplot(projection="3d")
x = np.linspace(190, 210, N)
y = np.linspace(990, 1010, N)
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, BeamStress(X, Y, b, h), alpha=0.8)
ax.set_facecolor("white")
plt.xlabel(r"$F$")
plt.ylabel(r"$L$");

# and so on ...  